In [15]:
import os
from email_bot import EmailSenderBot
import pandas as pd
from kafka import KafkaProducer
from tqdm import tqdm

In [16]:
"""
["../data/customer.csv","../data/date.csv","../data/product.csv","../data/reseller.csv","../data/sales.csv","../data/sales_order.csv","../data/sales_territory.csv"]
["prueba.customer","prueba.date","prueba.product","prueba.reseller","prueba.sales","prueba.sales-order","prueba.sales-terrritory"]
"""
DATA_INPUT_FILES = ["../data/date.csv"]#["../data/date.csv","../data/product.csv","../data/reseller.csv","../data/sales.csv","../data/sales_order.csv","../data/sales_territory.csv"]
KAFKA_SERVER = "20.49.8.224:9092"
KAFKA_TOPICS = ["prueba.date"]#["prueba.date","prueba.product","prueba.reseller","prueba.sales","prueba.sales-order","prueba.sales-terrritory"]

In [17]:
producer = KafkaProducer(bootstrap_servers = [KAFKA_SERVER])

In [18]:
sender_bot = EmailSenderBot()
sender_bot.set_mail_receptor("santiago.puerta@quind.io")

In [19]:
def send_2_kafka(input_data_path:str, kafka_topic_name:str) -> int:
    #me aseguro de que el archivo si existe
    assert os.path.exists(input_data_path), "El path al archivo de datos de entrada es erroneo, o el archivo no existe."
    file = os.path.splitext(input_data_path)
    #me aseguro de que la extension del archivo es CSV
    assert file[1] == ".csv", f"El archivo {file[0]} debe de ser de extension CSV."
    try:
        df = pd.read_csv(input_data_path)
        print(df.iloc[0:3])
        total_data = len(df) #total de datos a cargar en el topico
        progress_bar = tqdm(total=total_data, desc=f"Loading to {kafka_topic_name} topic")
        for _,row in df.iterrows():
            message = row.to_json()
            producer.send(kafka_topic_name, value=message.encode())
            producer.flush() #asegurarse de que todos los datos hayan sido escritos
            progress_bar.update(1)
        sender_bot.struct_message(
            email_subject=f"Topico: {kafka_topic_name} cargado exitosamente.",
            body = f"Se han cargado exitosamente {total_data} datos a el topico {kafka_topic_name} de kafka."
        )
        sender_bot.send()
        return 0
    except ValueError as error:
        sender_bot.struct_message(
            email_subject=f"Error cargando datos al topico {kafka_topic_name}.",
            body = f"Error: {error} cargando datos al topico {kafka_topic_name} de kafka."
        )
        sender_bot.send()
        print(error)
        return 1

In [20]:
def main():
    assert len(DATA_INPUT_FILES) == len(KAFKA_TOPICS), "Deben de haber tantos topicos como archivos de entrada hallan."
    for data_file, kafka_topic in zip(DATA_INPUT_FILES,KAFKA_TOPICS):
        send_2_kafka(
            input_data_path=data_file,
            kafka_topic_name=kafka_topic,
        )
    producer.close() #cierro conexion con el productor

In [21]:
if __name__ == "__main__":
    main()

    DateKey        Date FiscalYear FiscalQuarter     Month      FullDate  \
0  20170701  01/07/2017     FY2018     FY2018 Q1  2017 Jul  2017 Jul, 01   
1  20170702  02/07/2017     FY2018     FY2018 Q1  2017 Jul  2017 Jul, 02   
2  20170703  03/07/2017     FY2018     FY2018 Q1  2017 Jul  2017 Jul, 03   

   MonthKey  
0    201707  
1    201707  
2    201707  


Loading to prueba.date topic: 100%|██████████| 1461/1461 [02:01<00:00, 11.98it/s]
